# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.out** to Moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **run your notebook and keep all running logs** so that we can check.

## 1 $n$-gram Language Model
**Q1**: Expand the above definition of $ p(\vec{w})$ using naive estimates of the parameters, such as $  p(w_4 \mid w_2, w_3) \stackrel{\tiny{\mbox{def}}}{=}  \frac{C(w_2~w_3~w_4)}{C(w_2~w_3)} $ where \( C(w_2 w_3 w_4) \) denotes the count of times the trigram $ w_2 w_3 w_4 $ was observed in a training corpus.

**Write your answer:**

\# todo

$  p(\vec{w}) \stackrel{\tiny{\mbox{def}}}{=}
\frac{C(w_1)}{|\vec{w}|} \cdot
\frac{C(w_1~w_2)}{C(w_1)} \cdot
\frac{C(w_1~w_2~w_3)}{C(w_1~w_2)} \cdot
\frac{C(w_2~w_3~w_4)}{C(w_2~w_3)} \\ \cdots
\frac{C(w_n-2~w_n-1~w_n)}{C(w_n-1~w_n)}  $


**Q2**: One could also define a kind of reversed trigram language model $p_{reversed}$ that instead assumed the words were generated in reverse order (from right to left):
\begin{align} p_{reversed}(\vec{w}) \stackrel{\tiny{\mbox{def}}}{=}&p(w_n) \cdot p(w_{n-1} \mid w_n) \cdot p(w_{n-2} \mid w_{n-1} w_n) \cdot p(w_{n-3} \mid w_{n-2} w_{n-1}) \\ &\cdots p(w_2 \mid w_3 w_4) \cdot p(w_1 \mid w_2 w_3) \end{align}
By manipulating the notation, show that the two models are identical, i.e., $ p(\vec{w}) = p_{reversed}(\vec{w}) $ for any $ \vec{w} $ provided that both models use MLE parameters estimated from the same training data (see Q1 above).

**Write your answer:**

\# todo

$  p(\vec{w}) \stackrel{\tiny{\mbox{def}}}{=}
\frac{C(w_1)}{|\vec{w}|} \cdot
\frac{C(w_1~w_2)}{C(w_1)} \cdot
\frac{C(w_1~w_2~w_3)}{C(w_1~w_2)} \cdot
\frac{C(w_2~w_3~w_4)}{C(w_2~w_3)} \\ \cdots
\frac{C(w_n-2~w_n-1~w_n)}{C(w_n-2~w_n-1)}
\\ {=}
\frac{C(w_n)}{|\vec{w}|} \cdot
\frac{C(w_n~w_n-1)}{C(w_n)} \cdot
\frac{C(w_n~w_n-1~w_n-2)}{C(w_n~w_n-1)} \cdot
\frac{C(w_n-1~w_n-2~w_n-3)}{C(w_n-1~w_n-2)} \\ \cdots
\frac{C(w_3~w_2~w_1)}{C(w_3~w_2)}  $


## 2 $N$-gram Language Model Implementation

In [1]:
!wget -O train.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/train.txt
!wget -O dev.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/dev.txt
!wget -O test.txt https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/test.txt

--2023-10-20 21:19:01--  https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6640478 (6.3M) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]   6.33M  --.-KB/s    in 0.07s   

2023-10-20 21:19:02 (94.7 MB/s) - ‘train.txt’ saved [6640478/6640478]

--2023-10-20 21:19:02--  https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/lm/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Le

### 2.1 Building vocabulary

**Code**

\# todo


In [2]:
import numpy as np
from scipy.stats import gmean

import nltk
nltk.download("stopwords")
nltk.download('punkt')
nltk.download("stopwords")

from nltk import lm
from nltk.lm import preprocessing
import io
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords

from nltk.util import ngrams


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train = PlaintextCorpusReader("./","train.txt")
sents = train.sents()
# words = train.words()


# Add start-of-sentence tokens <s> and end-of-sentence </s> tokens.
uni_train,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_train,_  = preprocessing.padded_everygram_pipeline(2,sents)
tri_train,v = preprocessing.padded_everygram_pipeline(3,sents)

# Convert tokens that occur less than three times in the training data into a special unknown token <UNK>.
vocab = lm.Vocabulary(v,unk_cutoff=3)

# Print the vocabulary size.
print("Vocabulary size is:", len(vocab))

# for sent in train_text:
#     print(list(sent))

Vocabulary size is: 16948


In [ ]:
# from functools import partial
# def padded_ngram_pipeline(order, text):
#     padding_fn = partial(preprocessing.pad_both_ends, n=order)
#     return (
#         (ngrams(list(padding_fn(sent)), n=order) for sent in text),
#         preprocessing.flatten(map(padding_fn, text)),
    # )

**Discussion**

\# todo

There are 1 parameter in unigram model, 2 in bigram model, 3 in trigram model.


### 2.2 $N$-gram Language Modeling

**Code**

\# todo



In [4]:
uni_model = lm.MLE(1)
bi_model  = lm.MLE(2)
tri_model = lm.MLE(3)

uni_model.fit(uni_train,vocab)
bi_model.fit(bi_train,vocab)
# tri_model.fit(tri_train,vocab)

# len(bi_model.vocab)
# bi_model.vocab.lookup("</s>")

In [5]:
train = PlaintextCorpusReader("./","train.txt")
sents = train.sents()

uni_train,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_train,_  = preprocessing.padded_everygram_pipeline(2,sents)
# tri_train,_ = preprocessing.padded_everygram_pipeline(3,sents)

ppl = []
for sent in uni_train:
    ppl.append(uni_model.perplexity(list(sent)))
ppl_mean = gmean(ppl)
print("Perplexity of Unigram Model over train dataset:",ppl_mean)

ppl = []
for sent in bi_train:
    ppl.append(bi_model.perplexity(list(sent)))
ppl_mean = gmean(ppl)
print("Perplexity of Bigram Model over train dataset:",ppl_mean)

Perplexity of Unigram Model over train dataset: 977.7254663002035
Perplexity of Bigram Model over train dataset: 216.62822602581886


In [6]:
test  = PlaintextCorpusReader("./","dev.txt")
sents = test.sents()

uni_dev,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_dev,_  = preprocessing.padded_everygram_pipeline(2,sents)
tri_dev,_ = preprocessing.padded_everygram_pipeline(3,sents)

ppl = []
for sent in uni_dev:
    ppl.append(uni_model.perplexity(list(sent)))
ppl_mean = gmean(ppl)
print("Perplexity of Unigram Model over dev dataset:",ppl_mean)

ppl = []
for sent in bi_dev:
    ppl.append(bi_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print("Perplexity of Bigram Model over dev dataset:",ppl_mean)

Perplexity of Unigram Model over dev dataset: 945.1187134309084
Perplexity of Bigram Model over dev dataset: inf


**Discussion**

\# todo



### 2.3 Smoothing

#### 2.3.1 Add-one (Laplace) smoothing

**Code**

\# todo



In [7]:
train = PlaintextCorpusReader("./","train.txt")
sents = train.sents()

# uni_train,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_train,_  = preprocessing.padded_everygram_pipeline(2,sents)
# tri_train,_ = preprocessing.padded_everygram_pipeline(3,sents)

# uni_Laplace_model = lm.Laplace(1)
bi_Laplace_model  = lm.Laplace(2)
# tri_Laplace_model = lm.Laplace(3)

# uni_Laplace_model.fit(uni_train,vocab)
bi_Laplace_model.fit(bi_train,vocab)
# tri_Laplace_model.fit(tri_train,vocab)


In [8]:
train = PlaintextCorpusReader("./","train.txt")
sents = train.sents()

# uni_train,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_train,_  = preprocessing.padded_everygram_pipeline(2,sents)
# tri_train,_ = preprocessing.padded_everygram_pipeline(3,sents)

ppl = []
for sent in bi_train:
    ppl.append(bi_Laplace_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print("Perplexity of Bigram Model over train dataset:",ppl_mean)

Perplexity of Bigram Model over train dataset: 799.5663817970321


In [9]:
test  = PlaintextCorpusReader("./","dev.txt")
sents = test.sents()

# uni_dev,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_dev,_  = preprocessing.padded_everygram_pipeline(2,sents)
# tri_dev,_ = preprocessing.padded_everygram_pipeline(3,sents)

ppl = []
for sent in bi_dev:
    ppl.append(bi_Laplace_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print("Perplexity of Bigram Model over dev dataset:",ppl_mean)

Perplexity of Bigram Model over dev dataset: 885.955985029454


**Discussion**

\# todo



#### 2.3.2: Add-$k$ smoothing

**Code**

\# todo



In [10]:
train = PlaintextCorpusReader("./","train.txt")
sents = train.sents()

# uni_train,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_train,_  = preprocessing.padded_everygram_pipeline(2,sents)
# tri_train,_ = preprocessing.padded_everygram_pipeline(3,sents)
g = 0.5

# uni_Lidstone_model = lm.Lidstone(g,1,vocabulary=vocab)
bi_Lidstone_model  = lm.Lidstone(g,2,vocabulary=vocab)
# tri_Lidstone_model = lm.Lidstone(g,3,vocabulary=vocab)

# uni_Lidstone_model.fit(uni_train)
bi_Lidstone_model.fit(bi_train)
# tri_Lidstone_model.fit(tri_train)

test  = PlaintextCorpusReader("./","dev.txt")
sents = test.sents()

# uni_dev,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_dev,_  = preprocessing.padded_everygram_pipeline(2,sents)
# tri_dev,_ = preprocessing.padded_everygram_pipeline(3,sents)

ppl = []
for sent in bi_dev:
    ppl.append(bi_Lidstone_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print(f"Perplexity of Bigram Model over dev dataset when gamma is {g}:",ppl_mean)

Perplexity of Bigram Model over dev dataset when gamma is 0.5: 758.5314311570586


In [11]:
train = PlaintextCorpusReader("./","train.txt")
sents = train.sents()

# uni_train,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_train,_  = preprocessing.padded_everygram_pipeline(2,sents)
# tri_train,_ = preprocessing.padded_everygram_pipeline(3,sents)
g = 0.05

# uni_Lidstone_model = lm.Lidstone(g,1,vocabulary=vocab)
bi_Lidstone_model  = lm.Lidstone(g,2,vocabulary=vocab)
# tri_Lidstone_model = lm.Lidstone(g,3,vocabulary=vocab)

# uni_Lidstone_model.fit(uni_train)
bi_Lidstone_model.fit(bi_train)
# tri_Lidstone_model.fit(tri_train)

test  = PlaintextCorpusReader("./","dev.txt")
sents = test.sents()

# uni_dev,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_dev,_  = preprocessing.padded_everygram_pipeline(2,sents)
# tri_dev,_ = preprocessing.padded_everygram_pipeline(3,sents)

ppl = []
for sent in bi_dev:
    ppl.append(bi_Lidstone_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print(f"Perplexity of Bigram Model over dev dataset when gamma is {g}:",ppl_mean)

Perplexity of Bigram Model over dev dataset when gamma is 0.05: 512.8467604417024


In [12]:
train = PlaintextCorpusReader("./","train.txt")
sents = train.sents()

# uni_train,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_train,_  = preprocessing.padded_everygram_pipeline(2,sents)
# tri_train,_ = preprocessing.padded_everygram_pipeline(3,sents)
g = 0.01

# uni_Lidstone_model = lm.Lidstone(g,1,vocabulary=vocab)
bi_Lidstone_model  = lm.Lidstone(g,2,vocabulary=vocab)
# tri_Lidstone_model = lm.Lidstone(g,3,vocabulary=vocab)

# uni_Lidstone_model.fit(uni_train)
bi_Lidstone_model.fit(bi_train)
# tri_Lidstone_model.fit(tri_train)

test  = PlaintextCorpusReader("./","dev.txt")
sents = test.sents()

# uni_dev,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_dev,_  = preprocessing.padded_everygram_pipeline(2,sents)
# tri_dev,_ = preprocessing.padded_everygram_pipeline(3,sents)

ppl = []
for sent in bi_dev:
    ppl.append(bi_Lidstone_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print(f"Perplexity of Bigram Model over dev dataset when gamma is {g}:",ppl_mean)

Perplexity of Bigram Model over dev dataset when gamma is 0.01: 449.62736817084533


**Discussion**

\# todo



#### 2.3.3 Linear Interpolation

**Code**

\# todo



In [14]:
train = PlaintextCorpusReader("./","train.txt")
sents = train.sents()

uni_train,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_train,_  = preprocessing.padded_everygram_pipeline(2,sents)
tri_train,_ = preprocessing.padded_everygram_pipeline(3,sents)
a = 0.4

uni_StupidBackoff_model = lm.StupidBackoff(alpha=a,order=1,vocabulary=vocab)
bi_StupidBackoff_model  = lm.StupidBackoff(alpha=a,order=2,vocabulary=vocab)
tri_StupidBackoff_model = lm.StupidBackoff(alpha=a,order=3,vocabulary=vocab)

uni_StupidBackoff_model.fit(uni_train)
bi_StupidBackoff_model.fit(bi_train)
tri_StupidBackoff_model.fit(tri_train)

In [15]:
train = PlaintextCorpusReader("./","train.txt")
sents = train.sents()
uni_train,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_train,_  = preprocessing.padded_everygram_pipeline(2,sents)
tri_train,_ = preprocessing.padded_everygram_pipeline(3,sents)

# Unigram
ppl = []
for sent in uni_train:
    ppl.append(uni_StupidBackoff_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print(f"Perplexity of Unigram Model over train dataset when alpha is {a}:",ppl_mean)

# Bigram
ppl = []
for sent in bi_train:
    ppl.append(bi_StupidBackoff_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print(f"Perplexity of Bigram Model over train dataset when alpha is {a}:",ppl_mean)

# Trigram
ppl = []
for sent in tri_train:
    ppl.append(tri_StupidBackoff_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print(f"Perplexity of Trigram Model over train dataset when alpha is {a}:",ppl_mean)


Perplexity of Unigram Model over train dataset when alpha is 0.4: 977.7254663002035
Perplexity of Bigram Model over train dataset when alpha is 0.4: 216.62822602581886
Perplexity of Trigram Model over train dataset when alpha is 0.4: 57.53778400254985


In [16]:
test  = PlaintextCorpusReader("./","dev.txt")
sents = test.sents()
uni_dev,_ = preprocessing.padded_everygram_pipeline(1,sents)
bi_dev,_  = preprocessing.padded_everygram_pipeline(2,sents)
tri_dev,_ = preprocessing.padded_everygram_pipeline(3,sents)

# Unigram
ppl = []
for sent in uni_dev:
    ppl.append(uni_StupidBackoff_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print(f"Perplexity of Unigram Model over dev dataset when alpha is {a}:",ppl_mean)

# Bigram
ppl = []
for sent in bi_dev:
    ppl.append(bi_StupidBackoff_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print(f"Perplexity of Bigram Model over dev dataset when alpha is {a}:",ppl_mean)

# Trigram
ppl = []
for sent in tri_dev:
    ppl.append(tri_StupidBackoff_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print(f"Perplexity of Trigram Model over dev dataset when alpha is {a}:",ppl_mean)


Perplexity of Unigram Model over dev dataset when alpha is 0.4: 945.1187134309084
Perplexity of Bigram Model over dev dataset when alpha is 0.4: 338.8204045373383
Perplexity of Trigram Model over dev dataset when alpha is 0.4: 205.61093312112462


In [17]:
for aa in range(1,10,2):
    a = aa/10
    train = PlaintextCorpusReader("./","train.txt")
    sents = train.sents()

    uni_train,_ = preprocessing.padded_everygram_pipeline(1,sents)
    # bi_train,_  = preprocessing.padded_everygram_pipeline(2,sents)
    # tri_train,_ = preprocessing.padded_everygram_pipeline(3,sents)

    uni_StupidBackoff_model = lm.StupidBackoff(alpha=a,order=1,vocabulary=vocab)
    # bi_StupidBackoff_model  = lm.StupidBackoff(alpha=a,order=2,vocabulary=vocab)
    # tri_StupidBackoff_model = lm.StupidBackoff(alpha=a,order=3,vocabulary=vocab)

    uni_StupidBackoff_model.fit(uni_train)
    # bi_StupidBackoff_model.fit(bi_train)
    # tri_StupidBackoff_model.fit(tri_train)

    test  = PlaintextCorpusReader("./","dev.txt")
    sents = test.sents()
    uni_dev,_ = preprocessing.padded_everygram_pipeline(1,sents)
    # bi_dev,_  = preprocessing.padded_everygram_pipeline(2,sents)
    # tri_dev,_ = preprocessing.padded_everygram_pipeline(3,sents)

    ppl = []
    for sent in uni_dev:
        ppl.append(uni_StupidBackoff_model.perplexity(list(sent)))
    # ppl_mean = np.mean(ppl)
    ppl_mean = gmean(ppl)
    print(f"Perplexity of Unigram Model over dev dataset when alpha is {a}:",ppl_mean)

Perplexity of Unigram Model over dev dataset when alpha is 0.1: 945.1187134309084
Perplexity of Unigram Model over dev dataset when alpha is 0.3: 945.1187134309084
Perplexity of Unigram Model over dev dataset when alpha is 0.5: 945.1187134309084
Perplexity of Unigram Model over dev dataset when alpha is 0.7: 945.1187134309084
Perplexity of Unigram Model over dev dataset when alpha is 0.9: 945.1187134309084


In [18]:
for aa in range(1,10,2):
    a = aa/10
    train = PlaintextCorpusReader("./","train.txt")
    sents = train.sents()

    # uni_train,_ = preprocessing.padded_everygram_pipeline(1,sents)
    bi_train,_  = preprocessing.padded_everygram_pipeline(2,sents)
    # tri_train,_ = preprocessing.padded_everygram_pipeline(3,sents)

    # uni_StupidBackoff_model = lm.StupidBackoff(alpha=a,order=1,vocabulary=vocab)
    bi_StupidBackoff_model  = lm.StupidBackoff(alpha=a,order=2,vocabulary=vocab)
    # tri_StupidBackoff_model = lm.StupidBackoff(alpha=a,order=3,vocabulary=vocab)

    # uni_StupidBackoff_model.fit(uni_train)
    bi_StupidBackoff_model.fit(bi_train)
    # tri_StupidBackoff_model.fit(tri_train)

    test  = PlaintextCorpusReader("./","dev.txt")
    sents = test.sents()
    # uni_dev,_ = preprocessing.padded_everygram_pipeline(1,sents)
    bi_dev,_  = preprocessing.padded_everygram_pipeline(2,sents)
    # tri_dev,_ = preprocessing.padded_everygram_pipeline(3,sents)

    ppl = []
    for sent in bi_dev:
        ppl.append(bi_StupidBackoff_model.perplexity(list(sent)))
    # ppl_mean = np.mean(ppl)
    ppl_mean = gmean(ppl)
    print(f"Perplexity of Bigram Model over dev dataset when alpha is {a}:",ppl_mean)

Perplexity of Bigram Model over dev dataset when alpha is 0.1: 384.1621036225339
Perplexity of Bigram Model over dev dataset when alpha is 0.3: 347.7672326137989
Perplexity of Bigram Model over dev dataset when alpha is 0.5: 332.0395208219873
Perplexity of Bigram Model over dev dataset when alpha is 0.7: 322.0705270560666
Perplexity of Bigram Model over dev dataset when alpha is 0.9: 314.82035041825503


In [20]:
for aa in range(1,10,2):
    a = aa/10
    train = PlaintextCorpusReader("./","train.txt")
    sents = train.sents()

    # uni_train,_ = preprocessing.padded_everygram_pipeline(1,sents)
    # bi_train,_  = preprocessing.padded_everygram_pipeline(2,sents)
    tri_train,_ = preprocessing.padded_everygram_pipeline(3,sents)

    # uni_StupidBackoff_model = lm.StupidBackoff(alpha=a,order=1,vocabulary=vocab)
    # bi_StupidBackoff_model  = lm.StupidBackoff(alpha=a,order=2,vocabulary=vocab)
    tri_StupidBackoff_model = lm.StupidBackoff(alpha=a,order=3,vocabulary=vocab)

    # uni_StupidBackoff_model.fit(uni_train)
    # bi_StupidBackoff_model.fit(bi_train)
    tri_StupidBackoff_model.fit(tri_train)

    test  = PlaintextCorpusReader("./","dev.txt")
    sents = test.sents()
    # uni_dev,_ = preprocessing.padded_everygram_pipeline(1,sents)
    # bi_dev,_  = preprocessing.padded_everygram_pipeline(2,sents)
    tri_dev,_ = preprocessing.padded_everygram_pipeline(3,sents)

    ppl = []
    for sent in tri_dev:
        ppl.append(tri_StupidBackoff_model.perplexity(list(sent)))
    # ppl_mean = np.mean(ppl)
    ppl_mean = gmean(ppl)
    print(f"Perplexity of Trigram Model over dev dataset when alpha is {a}:",ppl_mean)

Perplexity of Trigram Model over dev dataset when alpha is 0.1: 301.7857806636453
Perplexity of Trigram Model over dev dataset when alpha is 0.3: 222.65360786315298
Perplexity of Trigram Model over dev dataset when alpha is 0.5: 193.29522779487237
Perplexity of Trigram Model over dev dataset when alpha is 0.7: 176.10525602129067
Perplexity of Trigram Model over dev dataset when alpha is 0.9: 164.2709241815869


In [21]:
train = PlaintextCorpusReader("./","train.txt")
sents = train.sents()

# uni_train,_ = preprocessing.padded_everygram_pipeline(1,sents)
# bi_train,_  = preprocessing.padded_everygram_pipeline(2,sents)
tri_train,_ = preprocessing.padded_everygram_pipeline(3,sents)
a = 0.9

# uni_StupidBackoff_model = lm.StupidBackoff(alpha=a,order=1,vocabulary=vocab)
# bi_StupidBackoff_model  = lm.StupidBackoff(alpha=a,order=2,vocabulary=vocab)
tri_StupidBackoff_model = lm.StupidBackoff(alpha=a,order=3,vocabulary=vocab)

# uni_StupidBackoff_model.fit(uni_train)
# bi_StupidBackoff_model.fit(bi_train)
tri_StupidBackoff_model.fit(tri_train)

test  = PlaintextCorpusReader("./","test.txt")
sents = test.sents()
# uni_test,_ = preprocessing.padded_everygram_pipeline(1,sents)
# bi_test,_  = preprocessing.padded_everygram_pipeline(2,sents)
tri_test,_ = preprocessing.padded_everygram_pipeline(3,sents)

ppl = []
for sent in tri_test:
    ppl.append(tri_StupidBackoff_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print(f"Perplexity of Trigram Model over test dataset when alpha is {a}:",ppl_mean)

Perplexity of Trigram Model over test dataset when alpha is 0.9: 161.4663188121758


**Discussion**

\# todo



##### **Optimization**:

\# todo

## 3 Preposition Prediction

In [26]:
!wget -O dev.in https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/prep/dev.in
!wget -O dev.out https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/prep/dev.out
!wget -O test.in https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/prep/test.in

--2023-10-20 21:37:57--  https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/prep/dev.in
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210427 (205K) [text/plain]
Saving to: ‘dev.in’

dev.in              100%[===================>] 205.50K  --.-KB/s    in 0.02s   

2023-10-20 21:37:57 (10.1 MB/s) - ‘dev.in’ saved [210427/210427]

--2023-10-20 21:37:57--  https://raw.githubusercontent.com/qtli/COMP7607-Fall2023/master/assignments/A1/data/prep/dev.out
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10

In [23]:
train = PlaintextCorpusReader("./","dev.in")
sents = train.sents()
f = open("./dd.out","w+")


for sent in sents:
    for i in range(len(sent)):
        if sent[i] == "PREP":
            # print(sent[i-2],sent[i+2])
            ans = tri_StupidBackoff_model.generate(text_seed=[sent[i-4],sent[i-3],sent[i-2]])
            # ans = tri_StupidBackoff_model.generate(text_seed=[sent[i-3],sent[i-2]])
            # ans = bi_StupidBackoff_model.generate(text_seed=[sent[i-3],sent[i-2]])
            # ans = bi_StupidBackoff_model.generate(text_seed=sent[i-2])
            num = 0
            while ans not in ["at", "in", "of", "for", "on"] and num <100:
                ans = tri_StupidBackoff_model.generate(text_seed=[sent[i-4],sent[i-3],sent[i-2]])
                # ans = tri_StupidBackoff_model.generate(text_seed=[sent[i-3],sent[i-2]])
                # ans = bi_StupidBackoff_model.generate(text_seed=[sent[i-3],sent[i-2]])
                # ans = bi_StupidBackoff_model.generate(text_seed=sent[i-2])
                num += 1
            print(ans,end=" ")
            f.write(str(ans)+" ")
    print("\n")
    f.write("\n")
f.close()

at of this at 

of 

of 

destroyed of on 

of in for on 

is for 

on 

of in in of of 

on of to 

of 

in 

of of of , 

in 

in for 

of in 

in of 

available of 

. of 

in at of 

of to at at <UNK> 

in in of 

in of of . 

into for of 

on in for of for 

at in of 

level 

at 

in in at of 

adulyadej in . 

of 

on for for 

being of 

on , 

so of 

in of 

of of 

in of 

outside 

in in 

in of in of 

on symptoms in 

of in 

on older in of 

in . friendly 

has 

, of to of 

on in financial of 

for 

of 

of of 

being of 

on 

of in with in it 

over of of of on 

in on 

in in 

in 

at for 

entered of in of 

in markets on 

in by 

in in in 

on in in 

of for will 

for in in 

in on for of 

, 

tuesday 

in 

in 

early 

on on 

of 

attacked of on reviewing 

on for in for 

on in on for 

in in 

on on 

of 

near sensor in on in 

searching - in on in on of at 

of for at for 

on of 

on of , 

of of in 

on of with of 

in of on 

of of 

of as 

in at o

In [24]:
import re
f = open("./dev.out","r")
gt = f.read()
gt = re.split(" |\n",gt)
gt = [g for g in gt if g != ""]
# print(len(gt))
print("Groundtruth:",end=" ")
print(gt)
f.close()

f = open("./dd.out","r")
p = f.read()
p = re.split(" |\n",p)

p = [pp for pp in p if pp != ""]
# print(len(p))
print("Prediction:",end=" ")
print(p)
f.close()

acc = (np.array(gt)==np.array(p)).mean()
print("Accuracy of prediction over dev dataset is",acc)


Groundtruth: ['on', 'of', 'in', 'in', 'of', 'of', 'in', 'of', 'for', 'of', 'in', 'for', 'on', 'at', 'in', 'on', 'of', 'in', 'in', 'on', 'of', 'on', 'on', 'in', 'on', 'on', 'of', 'on', 'of', 'in', 'in', 'for', 'in', 'of', 'on', 'in', 'of', 'on', 'of', 'in', 'of', 'on', 'for', 'of', 'of', 'at', 'on', 'at', 'on', 'in', 'of', 'of', 'of', 'in', 'on', 'on', 'on', 'for', 'of', 'in', 'in', 'at', 'of', 'for', 'on', 'for', 'of', 'on', 'at', 'of', 'on', 'in', 'of', 'of', 'in', 'on', 'at', 'on', 'on', 'in', 'in', 'of', 'on', 'on', 'in', 'of', 'in', 'of', 'on', 'of', 'on', 'of', 'in', 'in', 'in', 'at', 'of', 'at', 'of', 'on', 'of', 'in', 'of', 'on', 'for', 'for', 'in', 'of', 'on', 'at', 'on', 'on', 'on', 'of', 'for', 'in', 'of', 'in', 'on', 'of', 'on', 'of', 'of', 'for', 'in', 'of', 'on', 'of', 'in', 'for', 'in', 'in', 'at', 'of', 'in', 'of', 'in', 'in', 'on', 'in', 'in', 'in', 'at', 'on', 'at', 'of', 'in', 'of', 'in', 'on', 'in', 'in', 'for', 'for', 'of', 'of', 'at', 'in', 'at', 'of', 'on', 'in', 

In [27]:
test  = PlaintextCorpusReader("./","test.in")
sents = test.sents()

tri_test,_  = preprocessing.padded_everygram_pipeline(3,sents)


ppl = []
for sent in tri_test:
    ppl.append(tri_StupidBackoff_model.perplexity(list(sent)))
# ppl_mean = np.mean(ppl)
ppl_mean = gmean(ppl)
print(f"Perplexity of selected Trigram Model over test dataset:",ppl_mean)

Perplexity of selected Trigram Model over test dataset: 150.31106233064594


In [29]:
train = PlaintextCorpusReader("./","test.in")
sents = train.sents()
f = open("./3036198102.test.out","w+")


for sent in sents:
    for i in range(len(sent)):
        if sent[i] == "PREP":
            # print(sent[i-2],sent[i+2])
            ans = tri_StupidBackoff_model.generate(text_seed=[sent[i-4],sent[i-3],sent[i-2]])
            # ans = tri_StupidBackoff_model.generate(text_seed=[sent[i-3],sent[i-2]])
            # ans = bi_StupidBackoff_model.generate(text_seed=[sent[i-3],sent[i-2]])
            # ans = bi_StupidBackoff_model.generate(text_seed=sent[i-2])
            num = 0
            while ans not in ["at", "in", "of", "for", "on"] and num <100:
                ans = tri_StupidBackoff_model.generate(text_seed=[sent[i-4],sent[i-3],sent[i-2]])
                # ans = tri_StupidBackoff_model.generate(text_seed=[sent[i-3],sent[i-2]])
                # ans = bi_StupidBackoff_model.generate(text_seed=[sent[i-3],sent[i-2]])
                # ans = bi_StupidBackoff_model.generate(text_seed=sent[i-2])
                num += 1
            print(ans,end=" ")
            f.write(str(ans)+" ")
    print("\n")
    f.write("\n")
f.close()

in 

at in 

in 

of 

of of seven 

a 

for on 

for in on 

in at of to 

on in 

of in 

on for 

on 

a in on 

in 

routine on 

in of in of of 

and 

on of 

of on to deal 

on of to 

for of in 

of by 

in are on of on 

of selling 

on its of in 

on 

tuesday for 

of already 

for of 

of to in 

on at to 

of 

of for 

not in 

. 

traveled of 

in of <UNK> join 

in 

by 

on . in 

of in 

in in for for 

for 

in in of 

driver for 

in in 

of in as in 

in in 

, 

on of on from 

of a 

for research in 

built 

after in 

, of 

for in for of of 

, of of 

after in 

for 

for of of 

in 

from high of in 

of of 

on 

in 

in in of 

in of 

in 

in in 

for on 

in in tonnes of 

of their for 

of for for 

on basis 

is 

for 

from on for of 

of 

of of 

of of had 

of of 

on on of 

urged of for to on 

in at of 

of music in 

general for in of in 

will in of 

on in 

on 

, 

from 

of 

of on for world 

of ideas 

switch of and 

in ceasefire for , 